In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
device = "cuda"

In [20]:
class config:
    model = "resnet18"
    model1 = "resnet18"
    model2 = "resnet18"
    batchsize = 32
    WEIGHT_DECAY = 0.01
    EPOCHS = 15
class path:
    maindir = "/kaggle/input/gsoclensing/dataset/train"
    valid = "/kaggle/input/gsoclensing/dataset/val"

In [21]:
import torch
from tqdm import tqdm
import timm
import torch.nn as nn
from torchvision import transforms
import time
from torch.utils.data import DataLoader
import torch.nn.functional as F
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [22]:
imgtrans = transforms.Compose([
    transforms.ToPILImage(mode='L'),
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
]
)

In [29]:
def deltamat(shape):
    kk = np.zeros((shape,shape))
    for i in range(shape):
        for j in range(shape):
            kk[i][j] = (((i+1)**2)+((j+1)**2))**0.5
    return torch.tensor(kk,dtype = torch.float32)
positiontensor = deltamat(150)
positiontensor = positiontensor.unsqueeze(dim=0)
positiontensor=positiontensor.to("cuda")

In [30]:
class multiclass(nn.Module):
    def __init__(self):
        super(multiclass,self).__init__()
        self.resnet = timm.create_model(
            config.model,
            pretrained = True,
            drop_rate = 0.1
        )
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.resnet.fc = nn.Linear(in_features=512, out_features=3, bias=True)
    def forward(self,x):
        x = self.resnet(x)
        return x


In [31]:
class PINN1(nn.Module):
    def __init__(self,positionmat,batchsize = config.batchsize):
        super(PINN1,self).__init__()
        self.positionmat = positionmat
        self.batchsize = batchsize
        self.encoder = timm.create_model(
            config.model1,
            pretrained = True,
            drop_rate = 0.1
        )
        self.encoder.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.encoder.fc = nn.Linear(512,150*150,bias=True)
        
        self.doosramodel = timm.create_model(
            config.model2,
            pretrained = True,
            drop_rate = 0.1
        )
        self.doosramodel.conv1 = nn.Conv2d(2, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.doosramodel.fc = nn.Linear(in_features=512, out_features=3, bias=True)
    
    def lensingequation(self,angular,image):
        sourcedimage = image-angular
        return sourcedimage
    
    def forward(self,x):
        k = self.encoder(x)
        #print(k.shape)
        batch = k.shape[0]
        k = k.reshape(batch,1,150,150)
        angulardistort = k*self.positionmat
        #print(angulardistort.shape)
        sourced_image = self.lensingequation(x,angulardistort)
        twochannelimg = torch.cat((x,sourced_image),dim=1)
        features = self.doosramodel(twochannelimg)
        return features
        

In [32]:
class PINN2(nn.Module):
    def __init__(self,positionmat,batchsize = config.batchsize):
        super(PINN2,self).__init__()
        self.positionmat = positionmat
        self.batchsize = batchsize
        self.encoder = timm.create_model(
            config.model1,
            pretrained = True,
            drop_rate = 0.1
        )
        self.encoder.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.encoder.fc = nn.Linear(512,150*150,bias=True)
        
        self.doosramodel = timm.create_model(
            config.model2,
            pretrained = True,
            drop_rate = 0.1
        )
        self.doosramodel.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.doosramodel.fc = nn.Linear(in_features=512, out_features=3, bias=True)
    
    def lensingequation(self,angular,image):
        sourcedimage = image-angular
        return sourcedimage
    
    def forward(self,x):
        k = self.encoder(x)
        #print(k.shape)
        batch = k.shape[0]
        k = k.reshape(batch,1,150,150)
        angulardistort = k*self.positionmat
        #print(angulardistort.shape)
        sourced_image = self.lensingequation(x,angulardistort)
        #twochannelimg = torch.cat((x,sourced_image),dim=1)
        features = self.doosramodel(sourced_image)
        return features
        

In [34]:
# Define the model to load
modeltoload = "PINN1"
model = None

# Directory path to the weights
linktoweights = "directory_to_weights.pth"

if modeltoload == "multiclass":
    model = multiclass()
elif modeltoload == "PINN1":
    model = PINN1(positiontensor)  

if model is not None:
    # Loading the state dictionary of the model
    state_dict = torch.load(linktoweights)
    
    # Loading the state dictionary into the model
    model.load_state_dict(state_dict)
else:
    # Handle the case when the model type is not recognized
    print("Invalid model type specified.")

In [35]:
model

PINN1(
  (encoder): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act1): ReLU(inplace=True)
        (aa): Identity()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1),